In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337);

In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
chars = sorted(list(set(text)))
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string and output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# seed for generating text
idx = torch.tensor([0]).to(device)

In [6]:
# encoding the entire dataset into a tensor
data = torch.tensor(encode(text), dtype=torch.long)
# build dataset
n = int(0.9 * len(data)) # first 90% will be trian, rest val
train_data = data[:n]
val_data = data[n:]

In [7]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x[0], y[0]

In [8]:
eval_iters = 200
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out

In [9]:
# ------------------------------ Long Short-Term Memory -----------------------------------------------
class LSTM(nn.Module): 
    
    def __init__(self, hidden_size, output_size, embd_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.emb = nn.Embedding(output_size, embd_size) 
        self.linear = nn.Linear(embd_size, hidden_size)
        self.forget_gate = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Sigmoid(),
        )
        self.input_gate = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_size * 2, hidden_size),
                nn.Sigmoid()
            ),
            nn.Sequential(
                nn.Linear(hidden_size * 2, hidden_size),
                nn.Tanh()
            ),
        ])
        self.output_gate = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Sigmoid(),
        )
        self.tanh = nn.Tanh()
        self.lm_head = nn.Linear(hidden_size, output_size)

    def forward(self, X, y=None):
        # embed the characters
        X = self.emb(X)

        # reshape the embeddings
        X = self.linear(X) # (hidden_size, 1)

        # takes ht-1 and ct-1 and outputs h and c.
        def lstm_cell(c, h, x):
            
            # concatenate input to h
            h_x = torch.concat((h, x)) # (hidden_size +  hidden_size, 1)
            
            # forget gate
            fout = self.forget_gate(h_x) # (hidden_size, 1)
            # update c
            c = c * fout

            # input gate
            isig = self.input_gate[0](h_x) # (hidden_size, 1)
            itanh = self.input_gate[1](h_x) # (hidden_size, 1)
            iout = isig * itanh 
            # update c
            c = c + iout 

            # output gate    
            oout = self.output_gate(h_x) # (hidden_size, 1)
            h = self.tanh(c) * oout
            
            return c, h # (hidden_size, 1) both

        # initialize h and c to 0
        c = torch.zeros(self.hidden_size).to(device)
        h = torch.zeros(self.hidden_size).to(device)

        to_fill = torch.zeros(X.shape[0], self.hidden_size).to(device)

        # iterate through time
        for i, x in enumerate(X):
            c, h = lstm_cell(c, h, x)
            to_fill[i] = h
        
        # get logits
        logits = self.lm_head(to_fill)

        if y is None:
             return logits, None
            
        # calculate loss
        loss = F.cross_entropy(logits, y)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx, y=None)
            probs = F.softmax(logits, 0)
            idx_next = torch.multinomial(probs, num_samples=1)[0]
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next))
        return decode(idx.tolist())


In [10]:
# Hyperparameters-------
vocab_size = len(chars)
hidden_size = 512
embd_size = 192
batch_size = 1
block_size = 512
learning_rate = 3e-4
# ----------------------

In [11]:
model = LSTM(hidden_size=hidden_size, output_size=vocab_size, embd_size=embd_size)
model = model.to(device)
model.load_state_dict(torch.load('LSTM_model2.pt'))

<All keys matched successfully>

In [15]:
# training
max_steps = 50000
opt = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for steps in range(max_steps + 1):
    # sample batch data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model(xb, yb)
    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()
    if steps % (0.1 * max_steps) == 0:
        print(f'loss {loss.item()} / {steps/max_steps * 100:.2f}%')
    
    break

loss 0.9400046467781067 / 0.00%


In [13]:
print(estimate_loss())
# {'train': 1.3803791999816895, 'val': 1.599916934967041} 192, 82, 256
# {'train': 1.0186747312545776, 'val': 1.5402069091796875} 512, 192, 512

{'train': 1.0186747312545776, 'val': 1.5402069091796875}


In [42]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

2243841

In [52]:
# output of the model
print(model.generate(idx, 10))


3LJ3&ymAhB


In [13]:
@torch.no_grad()
def generate(idx, max_new_tokens):
        for i in range(max_new_tokens):
            logits, loss = model(idx, y=None)
            logits = logits[-1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next))
        return idx

In [16]:
# output of the model
print(decode(generate(idx, 512).tolist()))



KING EDWARD IV:
But, with those valiant prince of money; therefore none,
Were once my tongue upon all of all fault?

KING EDWARD IV:
Brother, and date up, Edward, did consul.
And yet--therefore comes the sea, and go.

HASTINGS:
Shall I see him or? What then?
First, more cause, God' love! I saw 'tis no something;
Might turn as I thus with more did sheep-loving mouths
Let on to meet my severing fair,
Or Richard: there is my subject, or did.

DUKE VINCENTIO:
Ha! keep your virtues a soul. Do serve me
To sorrow


In [19]:
torch.save(model.state_dict(), "LSTM_model3.pt")